# Foodnetwork 

### www.foodnetwork.com - a food recipe website with over 14,000 recipes.

This notebook performs webscraping on the foodnetwork website to collect

- AuthorName
- Recipe Title
- Average Recipe Rating
- Total number of Reviews
- Ingredients used in the Recipe
- Directions/Steps to cook the Recipe
- Nutrition content of the Recipe per Serving
- Time Required to Cook the Recipe

The above information is collected after having scraped the webpage links of all the recipes which is stored in the - foodNetworkRecipes dataFrame - which contains information such as the Recipe Title and it's corresponding Website Link

In [ ]:
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options

#### Author Name

In [ ]:
def get_authorName(doc):
    try:
        authorName_tag = doc.find_all("span", class_="o-Attribution__a-Name")[0]
        authorName = authorName_tag.find("a").text.strip()
        return authorName
    except:
        authorName = ''

#### Recipe Title

In [ ]:
def getRecipeTitle(doc):
    try:
        title = doc.find_all("h1")[0]
        recipeTitle = title.find_all("span")[0].text.strip()
        return recipeTitle
    except:
        pass

#### Average Recipe Rating & Review Count

In [ ]:
def get_avgRatingAndReviews(doc):
    try:
        averageRating_pattern = r'\d.\d'
        averageRating_of_recipe = doc.find("span", class_="sr-only").text
        averageRating_match = re.findall(averageRating_pattern, averageRating_of_recipe)
        averageRating_of_recipe = averageRating_match[0].strip()

        reviewCount_pattern = r'\d+'
        reviewCount = doc.find("h2", class_="reviews-ct").text
        reviewCount_match = re.findall(reviewCount_pattern, reviewCount)
        reviewCount = reviewCount_match[0].strip()
        
        return float(averageRating_of_recipe), int(reviewCount)
    except:
        pass

#### List of Ingredients used in the Recipe

In [ ]:
def get_ingredientUsedInRecipe(doc):
    try:
        ingredientsList = doc.find_all("span", class_="o-Ingredients__a-Ingredient--CheckboxLabel")
        ingredients = []
        for ingredient in ingredientsList:
            ingredients.append(ingredient.text)
        ingredients = ingredients[1:]
        return ingredients
    except:
        pass

#### Directions/Steps to cook the Recipe

In [ ]:
def get_recipeDirection(doc):
    try:
        directions = doc.find_all("li", class_="o-Method__m-Step")
        recipeSteps = {}
        step = 0
        for direction in directions:
            step += 1 
        #     print(step.text.strip(), '\n')
            recipeSteps[f'Step {step}'] = direction.text.strip()
        return recipeSteps
    except:
        pass

#### Nutrition Content of the Recipe per Serving

In [ ]:
def get_nutrition(doc):
    try:
        nutritionTable = doc.find("dl", class_="m-NutritionTable__a-Content")
        nutritionTitles = nutritionTable.find_all("dt", class_="m-NutritionTable__a-Headline")
        nutritionValues = nutritionTable.find_all("dd", class_="m-NutritionTable__a-Description")

        nutritionInfo = {}
        for title, value in zip(nutritionTitles, nutritionValues):
            nutritionInfo[title.string] = value.string
        return nutritionInfo
    except:
        pass

#### Time required to cook the Recipe

In [ ]:
def get_recipeTime(doc):
    recipeMaking_time_info = {}

    def addInfo(recipeMaking_time):
        # Iterate over <li> tags within <ul>
        for li_tag in recipeMaking_time.find_all('li'):
            # Get the headline and description spans within the current <li> tag
            headline = li_tag.find("span", class_="o-RecipeInfo__a-Headline").text.strip()
            description = li_tag.find("span", class_="o-RecipeInfo__a-Description").text.strip()

            # Add the key-value pair to the dictionary
            recipeMaking_time_info[headline[:-1]] = description
    try:
        for headlines in range(0, 3):
            recipeMaking_time = doc.find_all("ul", class_="o-RecipeInfo__m-Time")[headlines]
            addInfo(recipeMaking_time)
    except AttributeError:
        pass
    except IndexError:
        pass

    try:
        for entries in range(0, 3):
            recipeMaking_time = doc.find_all("ul", class_="o-RecipeInfo__m-Level")[entries]
            addInfo(recipeMaking_time)
    except AttributeError:
        pass
    except IndexError:
        pass
    
    return recipeMaking_time_info

In [ ]:
# Read the dataset that contains information about all the recipe's webpage links
foodNetworkRecipes_df = pd.read_csv('./foodNetworkRecipeLinks.csv')

#### Iterate through all the recipeLinks on Food Network and scrape the data

In [ ]:
start_time = time.time()

recipeDetails = []
columns = ['Recipe Title', 'Author Name', 'Average Rating of the Recipe', 'Number of Reviews', 'Ingredient(s)', 'Recipe Step(s)', 'Nutrition Information', 'Time to Cook the Recipe', 'Recipe Page Link']

chrome_options = Options()
chrome_options.add_argument("--headless")
driver = Chrome(options = chrome_options)

count = 0

for recipeLink in foodNetworkRecipes_df['Recipe Link']:

    url = recipeLink
    
    driver.get(url)
    html_content = driver.page_source
    doc = BeautifulSoup(html_content, 'html.parser')
    
    # suspend execution for 1 second
    time.sleep(1)
    
    try:
        recipeTitle = getRecipeTitle(doc)
        authorName = get_authorName(doc)
        averageRating_of_recipe, reviewCount = get_avgRatingAndReviews(doc)
        ingredients = get_ingredientUsedInRecipe(doc)
        recipeSteps = get_recipeDirection(doc)
        nutritionInfo = get_nutrition(doc)
        timeToCookRecipe = get_recipeTime(doc)
        recipePageLink = recipeLink

        recipeDetails.append((recipeTitle, authorName, averageRating_of_recipe, reviewCount, ingredients, recipeSteps, nutritionInfo, timeToCookRecipe, recipePageLink))
        
        # Periodically save the scraped recipes to system - in the case of a crash
        count += 1
        
        if ((count % 500) == 0):
            foodNetwork_df = pd.DataFrame(recipeDetails, columns = columns)
            foodNetwork_df.to_csv('./Food Network Recipes.csv', index = None)
        else:
            continue
    
    except TypeError:
        print('-------------------------------------------------------')
        print('Recipe Title: ', recipeTitle)
        print('Recipe Link: ', recipePageLink)
        print(f'Average Rating: {averageRating_of_recipe} Review Count = {reviewCount}')
        
driver.quit()
end_time = time.time()

In [ ]:
foodNetwork_df = pd.DataFrame(recipeDetails, columns = columns)

total_time = end_time - start_time
print(f'Total Time = {total_time} seconds')

foodNetwork_df.to_csv('./Food Network Recipes.csv', index = None)